In [2]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load('/home/user01/data/talha/CWD/new_model.onnx') 

In [4]:
# Import the ONNX model to Tensorflow
tf_rep = prepare(onnx_model)

2023-06-07 16:38:37.519985: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
tf_rep.export_graph('/home/user01/data/talha/CWD/new_model.pb')

2023-06-07 16:41:27.413393: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor '2902' with dtype float and shape [1280]
	 [[{{node 2902}}]]


INFO:tensorflow:Assets written to: /home/user01/data/talha/CWD/new_model.pb/assets


2023-06-07 16:41:51.730233: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_input' with dtype float and shape [?,3,224,224]
	 [[{{node serving_default_input}}]]
INFO:tensorflow:Assets written to: /home/user01/data/talha/CWD/new_model.pb/assets


In [52]:
import tensorflow as tf
import numpy as np
import json, requests
import cv2
import numpy as np

# Load the model
loaded_model = tf.saved_model.load('/home/user01/data/talha/CWD/new_model.pb')
# Predict
# Here 'serving_default' is the name of the serving signature and 'input' is the name of the input tensor. 
# These names might be different for your model. You can print out `loaded_model.signatures` to see the names in your model.
print(loaded_model.signatures)
# laod labesl
imagenet_labels_url = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
labels = json.loads(requests.get(imagenet_labels_url).text)

img = cv2.imread('/home/user01/data/talha/CWD/s.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224,224))
img = img /255

img = img[np.newaxis, ...]
img = img.transpose(0,3,1,2)
img = tf.convert_to_tensor(img)
img = tf.cast(img, tf.float32)
print(img.shape)

predictions = loaded_model.signatures['serving_default'](input=img)
preds = predictions['output']
preds = preds.numpy()
predicted_index = np.argmax(preds, axis=-1)[0]

# Map the predicted class index to its corresponding label
predicted_class = labels[predicted_index]

print(predicted_class)

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, input) at 0x7F4DD82CC1F0>})
(1, 3, 224, 224)
strawberry


In [54]:
import tensorflow as tf


# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model('/home/user01/data/talha/CWD/new_model.pb')
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open("/home/user01/data/talha/CWD/model.tflite", "wb") as f:
    f.write(tflite_model)

2023-06-07 17:34:20.900627: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_input' with dtype float and shape [?,3,224,224]
	 [[{{node serving_default_input}}]]
2023-06-07 17:34:20.931766: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-07 17:34:20.931790: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-07 17:34:20.931938: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/user01/data/talha/CWD/new_model.pb
2023-06-07 17:34:20.936904: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-07 17:34:20.936925: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /home/user01/

In [67]:
import numpy as np
from pprint import pprint
import tensorflow as tf

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="/home/user01/data/talha/CWD/model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors information
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

pprint(input_details)
pprint(output_details)

# Prepare your input data corresponding to the input tensor shape.
img = cv2.imread('/home/user01/data/talha/CWD/c.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224,224))
img = img /255

img = img[np.newaxis, ...]
img = img.transpose(0,3,1,2)
img = tf.convert_to_tensor(img)
img = tf.cast(img, tf.float32)
input_data = img

# Set the tensor to point to your input data
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the model
interpreter.invoke()

# Retrieve the output of the model
output_data = interpreter.get_tensor(output_details[0]['index'])

predicted_index = np.argmax(output_data, axis=-1)[0]

# Map the predicted class index to its corresponding label
predicted_class = labels[predicted_index]

print(predicted_class)

[{'dtype': <class 'numpy.float32'>,
  'index': 0,
  'name': 'serving_default_input:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([], dtype=float32),
                              'zero_points': array([], dtype=int32)},
  'shape': array([  1,   3, 224, 224], dtype=int32),
  'shape_signature': array([ -1,   3, 224, 224], dtype=int32),
  'sparsity_parameters': {}}]
[{'dtype': <class 'numpy.float32'>,
  'index': 275,
  'name': 'PartitionedCall:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([], dtype=float32),
                              'zero_points': array([], dtype=int32)},
  'shape': array([   1, 1000], dtype=int32),
  'shape_signature': array([  -1, 1000], dtype=int32),
  'sparsity_parameters': {}}]
Egyptian Mau


In [68]:
tf.__version__

'2.12.0'

In [69]:
onnx.__version__

'1.14.0'

In [71]:
import onnx_tf

In [72]:
onnx_tf.__version__

'1.10.0'